In [9]:
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv

# Securely retrieve API key from Streamlit secrets
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")




In [12]:
# Load and process PDF documents, caching only this step
@st.cache_data
def load_and_process_pdfs(pdf_folder_path):
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    return splits

2024-11-10 17:35:06.008 No runtime found, using MemoryCacheStorageManager


In [15]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

try:
    openai.Model.list()  # This makes a small request to check connection
    print("Connected to OpenAI API.")
except Exception as e:
    print(f"Connection error: {e}")


Connection error: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [16]:
import openai
from dotenv import load_dotenv
import os

# Load the API key from the .env file
load_dotenv()
api_key = os.getenv("API_KEY")
openai.api_key = api_key

# Test API connection by making a small call
try:
    # Instead of listing models, try a basic call with the latest interface
    response = openai.Completion.create(
        model="text-ada-001",
        prompt="Test",
        max_tokens=5
    )
    print("Connected to OpenAI API. Response:", response.choices[0].text)
except Exception as e:
    print(f"Connection error: {e}")


Connection error: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [13]:
# Load and process documents
pdf_folder_path = "./fin_ed_docs"
splits = load_and_process_pdfs(pdf_folder_path)

# Initialize vector store without caching
def initialize_vectorstore(splits):
    embeddings = OpenAIEmbeddings(api_key=api_key)
    return FAISS.from_documents(documents=splits, embedding=embeddings)

# Initialize vector store
vectorstore = initialize_vectorstore(splits)

2024-11-10 17:35:14.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:35:14.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:35:14.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:35:14.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:35:14.060 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


APIConnectionError: Connection error.

In [ ]:

# Prompt template for LLM
prompt_template = """You are a finance expert. You need to answer the question related to finance. 
Given below is the context and question of the user. Don't answer question outside the context provided.
context = {context}
question = {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=api_key)

# Helper function to format document content
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Set up the RAG (Retrieval-Augmented Generation) chain
rag_chain = (
    {"context": vectorstore.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Streamlit app
st.title("Finance Expert")
user_input = st.text_input("Enter your question about finance:", "")
if st.button("Submit"):
    try:
        response = rag_chain.invoke(user_input)
        st.write(response)
    except Exception as e:
        st.write(f"An error occurred: {e}")
